- word2vec
- 크롤링 -> 데이터 전처리(->csv) -> 전처리한 csv에 text를 토크나이즈 -> vocab -> embedding -> 모델(RNN이나 CNN) 학습 -> 예측값
-     시명님, 진우님 완료                앞 뒤로 불용어 제거

#### 각 왕 시대에 가장 많이 언급된 이름, 혹은 관직 아니면 기관
- word2vec로 왕과 언급된 이름의 연관도를 예측 %로 나타내기
- 이름이 없으면 관직, 관직이 없으면 기관
- gpt의 word2vec 코드
- 데이터 -> 파일 -> 토크나이징 -> vocab -> embedding ~ embedding bag -> model
-                                           (라벨링)             (word2vec)

- 각 세대 왕의 실록의 단어 빈도를 체크 -> 그 중에서 언급된 이름 중 가장 많이 언급된 이름을 선택 -> 그 이름과 왕과 연관이 있는지 유사도 %를 나타내보기
- ex) 태조를 적으면 정도전이 %나오는지 확인
- 결론: 그 세대의 왕과 그 단어(이름, 관직, 기관)가 얼마나 연관되어있는지 퍼센트를 보고 역사적 기록을 그 만한 비중인지 확인해보기(맞는지 아닌지)

In [1092]:
# 모듈 로딩
import re
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from collections import Counter
from konlpy.tag import Okt

In [1093]:
# 데이터 확인
trainDF = pd.read_table('./data/KING/king_20.csv', encoding='utf-8', sep=';')
trainDF

,label,year,month,day,text
0,20,10년,10월,10일,"문신(文臣)과 종신(宗臣)에게 사후(射帿)573) 하기를 명하였는데, 문신으로 활..."
1,20,10년,10월,11일,"지평(持平) 이성효(李性孝)가 상소했는데, 대략 이르기를, ""전하께서 10년 동안..."
2,20,10년,10월,12일,"이보욱(李普昱)을 사간으로, 임정(任珽)을 교리로, 정형복(鄭亨復)을 부교리로, ..."
3,20,10년,10월,13일,"정필녕(鄭必寧)을 승지로, 민응수(閔應洙)를 부제학으로 삼았다."
4,20,10년,10월,13일,"원계군(原溪君) 엽(㮿)이 상소하였는데, 대략 이르기를, ""고(故) 종신(宗臣) ..."
...,...,...,...,...,...
36716,20,즉위년,9월,5일,이보다 앞서 함경 감사(咸鏡監司) 이의만(李宜晩)이 본도(本道)의 살옥(殺獄)에 대...
36717,20,즉위년,9월,6일,"대사헌(大司憲) 이명언(李明彦)이 말하기를, ""현종 대왕(顯宗大王)의 상(喪) 때..."
36718,20,즉위년,9월,7일,이때에 임금이 감기를 앓은 지 오래 되어도 낫지 아니하였는데도 오히려 아침 저녁의 ...
36719,20,즉위년,9월,7일,임금이 사관(史官)을 보내어 찬선(贊善) 정제두(鄭齊斗)를 유소(諭召)하여 함께 오...


In [1094]:
# 불필요 컬럼 삭제
trainDF.drop(columns=['year', 'month', 'day'], inplace=True)
print(f'trainDF => {trainDF.shape}')

trainDF => (36721, 2)


In [1095]:
# 클래스 비율 체크
trainDF['label'].value_counts()

label
20    36721
Name: count, dtype: int64

In [1096]:
# 피쳐와 레이블 분리
print(f'colnumns => {trainDF.columns}')

textDF = trainDF.text.to_frame()
labelSR=trainDF.label

print(f'textDF => {textDF.shape}, labelSR => {labelSR.shape}')

colnumns => Index(['label', 'text'], dtype='object')
textDF => (36721, 1), labelSR => (36721,)


In [1097]:
# 한글과 공백을 제외하고 모두 제거
textDF['text'] = textDF['text'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '')
textDF[:5]

,text
0,"문신(文臣)과 종신(宗臣)에게 사후(射帿)573) 하기를 명하였는데, 문신으로 활..."
1,"지평(持平) 이성효(李性孝)가 상소했는데, 대략 이르기를, ""전하께서 10년 동안..."
2,"이보욱(李普昱)을 사간으로, 임정(任珽)을 교리로, 정형복(鄭亨復)을 부교리로, ..."
3,"정필녕(鄭必寧)을 승지로, 민응수(閔應洙)를 부제학으로 삼았다."
4,"원계군(原溪君) 엽(㮿)이 상소하였는데, 대략 이르기를, ""고(故) 종신(宗臣) ..."


In [1098]:
# 불용어 정의
stop_words = './data/stop_word.txt'

with open(stop_words, encoding='utf-8') as f:
    lines = f.read().splitlines()

stopwords = set(lines) # set을 이용하여 혹시 모를 중복을 피함

# 형태소 분석기 Okt를 사용한 토큰화 작업
okt = Okt()

tokenized_data = []
for sentence in textDF['text']:
    tokenized_sentence = okt.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if word not in stopwords and not re.match(r'[^ 가-힣]+$', word)] # 불용어와 한자 제거
    tokenized_data.append(stopwords_removed_sentence)
    #tokenized_data.extend(stopwords_removed_sentence) # extend를 사용하여 리스트를 확장합니다. 표 만들 때 사용

In [1099]:
# # 빈도 수 시각화
# word_counts = Counter(tokenized_data)

# # 상위 10개 단어 추출
# top_words = dict(word_counts.most_common(10))

# # 막대 그래프 그리기
# plt.figure(figsize=(8, 4))
# plt.bar(top_words.keys(), top_words.values(), color='red')
# plt.xlabel('단어')
# plt.ylabel('빈도수')
# plt.title('세조 시대 단어 빈도 상위 10개')
# plt.xticks()
# plt.show()

In [1100]:
# sg=1 skip-gram: 입력을 받아서 주변 단어를 예측
# sg=0 CBoW: 주변에 있는 단어를 가지고 중간에 있는 단어를 예측
model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4, sg=1, epochs=10)

2024-04-05 12:25:47,784 : INFO : collecting all words and their counts
2024-04-05 12:25:47,784 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-04-05 12:25:47,916 : INFO : PROGRESS: at sentence #10000, processed 745822 words, keeping 43232 word types
2024-04-05 12:25:48,025 : INFO : PROGRESS: at sentence #20000, processed 1390881 words, keeping 59007 word types
2024-04-05 12:25:48,121 : INFO : PROGRESS: at sentence #30000, processed 1883414 words, keeping 67065 word types
2024-04-05 12:25:48,214 : INFO : collected 75871 word types from a corpus of 2443565 raw words and 36721 sentences
2024-04-05 12:25:48,214 : INFO : Creating a fresh vocabulary
2024-04-05 12:25:48,512 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 75871 unique words (100.00% of original 75871, drops 0)', 'datetime': '2024-04-05T12:25:48.512938', 'gensim': '4.3.2', 'python': '3.8.19 (default, Mar 20 2024, 19:55:45) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Window

In [1101]:
model.wv.vectors.shape

(75871, 100)

In [1102]:
model.wv.most_similar("영조")

[('어계선', 0.7416821122169495),
 ('함격', 0.7394095659255981),
 ('가로막아야', 0.7360052466392517),
 ('뽑혔고', 0.7337253093719482),
 ('였었고', 0.7290489673614502),
 ('되었었으므로', 0.7288016676902771),
 ('종정', 0.7287483215332031),
 ('가리킴', 0.7188725471496582),
 ('올려주었고', 0.7186225652694702),
 ('차천로', 0.7170872092247009)]

In [1103]:
print(model.wv.similarity("영조", "이인좌"))

0.35901135
